In [106]:
import pandas as pd
import seaborn as sns
import numpy as np

Оставим и сгенерируем признаки, которые мы знаем до следующих игр, т.е если мы будем брать количество пассов на каждую игру для обучения, то при прогнозирование будущей игры мы эти данные не знаем.
Оставим сезоны начиная с 2018, 2023 уходит на валидацию, поэтому при создании признаков не учитываем этот сезон

In [107]:
n = 100 #количество матчей для валидации

pd.set_option('display.max_columns', None)
df = pd.read_csv('../../../EDA/official/eda_data/games.csv')
df_club = pd.read_csv('../../../EDA/official/eda_data/clubs_stat.csv')
df.head()

#df = df.iloc #датасет для агрегации, 80 последних матчей - для валидации, их исключаем чтобы не было утечки данных
df.head()

,match_id,gameweek_id,gameweek_compSeason_label,gameweek_compSeason_id,gameweek_gameweek,kickoff_millis,kickoff_label,provisionalKickoff_millis,provisionalKickoff_label,teams_team_1_name,teams_team_1_club_abbr,teams_team_1_club_id,teams_score_1,teams_team_2_name,teams_team_2_club_abbr,teams_team_2_club_id,teams_score_2,ground_name,ground_city,ground_id,outcome,attendance,clock_secs,clock_label,behindClosedDoors,id,team_1_formation_used,team_1_accurate_back_zone_pass,team_1_total_back_zone_pass,team_1_accurate_pass,team_1_touches,team_1_total_pass,team_1_backward_pass,team_1_successful_open_play_pass,team_1_open_play_pass,team_1_possession_percentage,team_1_leftside_pass,team_1_accurate_fwd_zone_pass,team_1_rightside_pass,team_1_total_long_balls,team_1_total_fwd_zone_pass,team_1_touches_in_opp_box,team_1_final_third_entries,team_1_crosses_18yard,team_1_successful_final_third_passes,team_1_accurate_cross,team_1_long_pass_own_to_opp,team_1_fwd_pass,team_1_total_cross_nocorner,team_1_total_cross,team_1_total_final_third_passes,team_1_long_pass_own_to_opp_success,team_1_poss_lost_all,team_1_poss_lost_ctrl,team_1_pen_area_entries,team_1_passes_left,team_1_accurate_long_balls,team_1_crosses_18yardplus,team_1_accurate_cross_nocorner,team_1_attempts_ibox,team_1_total_scoring_att,team_1_att_openplay,team_1_att_lf_target,team_1_att_bx_centre,team_1_ontarget_scoring_att,team_1_ball_recovery,team_1_att_lf_total,team_1_poss_won_att_3rd,team_1_att_ibox_target,team_1_ontarget_att_assist,team_1_blocked_pass,team_1_total_att_assist,team_1_duel_won,team_1_won_contest,team_1_att_sv_low_centre,team_1_successful_put_through,team_1_total_contest,team_1_att_assist_openplay,team_1_put_through,team_1_poss_won_mid_3rd,team_1_total_tackle,team_1_interception,team_1_interception_won,team_1_won_tackle,team_1_total_offside,team_1_duel_lost,team_1_accurate_layoffs,team_1_passes_right,team_1_total_chipped_pass,team_1_total_layoffs,team_1_accurate_chipped_pass,team_1_aerial_lost,team_1_effective_clearance,team_1_lost_corners,team_1_total_launches,team_1_total_clearance,team_1_attempts_conceded_ibox,team_1_fk_foul_won,team_1_poss_won_def_3rd,team_1_saved_ibox,team_1_saves,team_1_diving_save,team_1_challenge_lost,team_1_dispossessed,team_1_unsuccessful_touch,team_1_goal_kicks,team_1_accurate_goal_kicks,team_1_attempts_conceded_obox,team_1_head_clearance,team_1_effective_head_clearance,team_1_fk_foul_lost,team_1_attempted_tackle_foul,team_1_aerial_won,team_1_goals_conceded_ibox,team_1_goals_conceded,team_1_total_yel_card,team_1_shot_off_target,team_1_total_throws,team_1_att_rf_total,team_1_accurate_throws,team_1_att_ibox_miss,team_1_offtarget_att_assist,team_1_saved_obox,team_1_outfielder_block,team_1_att_rf_target,team_1_corner_taken,team_1_won_corners,team_1_accurate_corners_intobox,team_1_att_assist_setplay,team_1_att_miss_right,team_1_att_corner,team_1_total_corners_intobox,team_1_att_hd_total,team_1_att_hd_miss,team_1_effective_blocked_cross,team_1_blocked_cross,team_1_subs_made,team_1_accurate_launches,team_1_fouled_final_third,team_1_att_obx_centre,team_1_attempts_obox,team_1_att_obox_target,team_1_keeper_throws,team_1_accurate_keeper_throws,team_1_total_through_ball,team_1_big_chance_missed,team_1_big_chance_created,team_1_att_miss_left,team_1_blocked_scoring_att,team_1_att_ibox_blocked,team_1_total_flick_on,team_1_losses,team_2_formation_used,team_2_open_play_pass,team_2_leftside_pass,team_2_accurate_goal_kicks,team_2_possession_percentage,team_2_poss_lost_all,team_2_goal_kicks,team_2_poss_lost_ctrl,team_2_total_pass,team_2_total_long_balls,team_2_total_back_zone_pass,team_2_total_chipped_pass,team_2_successful_open_play_pass,team_2_accurate_long_balls,team_2_accurate_back_zone_pass,team_2_rightside_pass,team_2_unsuccessful_touch,team_2_accurate_pass,team_2_accurate_chipped_pass,team_2_touches,team_2_attempts_conceded_ibox,team_2_saves,team_2_saved_ibox,team_2_ball_recovery,team_2_challenge_lost,team_2_backward_pass,team_2_successful_final_third_passes,t

In [108]:
home_games = df[['gameweek_compSeason_label', 'teams_team_1_name']].rename(columns={'gameweek_compSeason_label':'Season', 'teams_team_1_name': 'Team'})
away_games = df[['gameweek_compSeason_label', 'teams_team_2_name']].rename(columns={'gameweek_compSeason_label':'Season','teams_team_2_name': 'Team'})

all_games = pd.concat([home_games, away_games])
all_games['Games']=1

result = all_games.groupby(['Season', 'Team']).agg({'Games': 'sum'}).reset_index()
result

,Season,Team,Games
0,2019,Arsenal,38
1,2019,Aston Villa,38
2,2019,Bournemouth,38
3,2019,Brighton & Hove Albion,38
4,2019,Burnley,38
...,...,...,...
95,2023,Nottingham Forest,20
96,2023,Sheffield United,20
97,2023,Tottenham Hotspur,20
98,2023,West Ham United,20


team_1_hue - target (Выиграла команда 1 -1, проиграла -0, ничья - 2)

In [109]:
df = df[df['gameweek_compSeason_label'] >= 2018]
df_club = df_club[df_club['season'] >= 2018]

In [110]:
df.head()

,match_id,gameweek_id,gameweek_compSeason_label,gameweek_compSeason_id,gameweek_gameweek,kickoff_millis,kickoff_label,provisionalKickoff_millis,provisionalKickoff_label,teams_team_1_name,teams_team_1_club_abbr,teams_team_1_club_id,teams_score_1,teams_team_2_name,teams_team_2_club_abbr,teams_team_2_club_id,teams_score_2,ground_name,ground_city,ground_id,outcome,attendance,clock_secs,clock_label,behindClosedDoors,id,team_1_formation_used,team_1_accurate_back_zone_pass,team_1_total_back_zone_pass,team_1_accurate_pass,team_1_touches,team_1_total_pass,team_1_backward_pass,team_1_successful_open_play_pass,team_1_open_play_pass,team_1_possession_percentage,team_1_leftside_pass,team_1_accurate_fwd_zone_pass,team_1_rightside_pass,team_1_total_long_balls,team_1_total_fwd_zone_pass,team_1_touches_in_opp_box,team_1_final_third_entries,team_1_crosses_18yard,team_1_successful_final_third_passes,team_1_accurate_cross,team_1_long_pass_own_to_opp,team_1_fwd_pass,team_1_total_cross_nocorner,team_1_total_cross,team_1_total_final_third_passes,team_1_long_pass_own_to_opp_success,team_1_poss_lost_all,team_1_poss_lost_ctrl,team_1_pen_area_entries,team_1_passes_left,team_1_accurate_long_balls,team_1_crosses_18yardplus,team_1_accurate_cross_nocorner,team_1_attempts_ibox,team_1_total_scoring_att,team_1_att_openplay,team_1_att_lf_target,team_1_att_bx_centre,team_1_ontarget_scoring_att,team_1_ball_recovery,team_1_att_lf_total,team_1_poss_won_att_3rd,team_1_att_ibox_target,team_1_ontarget_att_assist,team_1_blocked_pass,team_1_total_att_assist,team_1_duel_won,team_1_won_contest,team_1_att_sv_low_centre,team_1_successful_put_through,team_1_total_contest,team_1_att_assist_openplay,team_1_put_through,team_1_poss_won_mid_3rd,team_1_total_tackle,team_1_interception,team_1_interception_won,team_1_won_tackle,team_1_total_offside,team_1_duel_lost,team_1_accurate_layoffs,team_1_passes_right,team_1_total_chipped_pass,team_1_total_layoffs,team_1_accurate_chipped_pass,team_1_aerial_lost,team_1_effective_clearance,team_1_lost_corners,team_1_total_launches,team_1_total_clearance,team_1_attempts_conceded_ibox,team_1_fk_foul_won,team_1_poss_won_def_3rd,team_1_saved_ibox,team_1_saves,team_1_diving_save,team_1_challenge_lost,team_1_dispossessed,team_1_unsuccessful_touch,team_1_goal_kicks,team_1_accurate_goal_kicks,team_1_attempts_conceded_obox,team_1_head_clearance,team_1_effective_head_clearance,team_1_fk_foul_lost,team_1_attempted_tackle_foul,team_1_aerial_won,team_1_goals_conceded_ibox,team_1_goals_conceded,team_1_total_yel_card,team_1_shot_off_target,team_1_total_throws,team_1_att_rf_total,team_1_accurate_throws,team_1_att_ibox_miss,team_1_offtarget_att_assist,team_1_saved_obox,team_1_outfielder_block,team_1_att_rf_target,team_1_corner_taken,team_1_won_corners,team_1_accurate_corners_intobox,team_1_att_assist_setplay,team_1_att_miss_right,team_1_att_corner,team_1_total_corners_intobox,team_1_att_hd_total,team_1_att_hd_miss,team_1_effective_blocked_cross,team_1_blocked_cross,team_1_subs_made,team_1_accurate_launches,team_1_fouled_final_third,team_1_att_obx_centre,team_1_attempts_obox,team_1_att_obox_target,team_1_keeper_throws,team_1_accurate_keeper_throws,team_1_total_through_ball,team_1_big_chance_missed,team_1_big_chance_created,team_1_att_miss_left,team_1_blocked_scoring_att,team_1_att_ibox_blocked,team_1_total_flick_on,team_1_losses,team_2_formation_used,team_2_open_play_pass,team_2_leftside_pass,team_2_accurate_goal_kicks,team_2_possession_percentage,team_2_poss_lost_all,team_2_goal_kicks,team_2_poss_lost_ctrl,team_2_total_pass,team_2_total_long_balls,team_2_total_back_zone_pass,team_2_total_chipped_pass,team_2_successful_open_play_pass,team_2_accurate_long_balls,team_2_accurate_back_zone_pass,team_2_rightside_pass,team_2_unsuccessful_touch,team_2_accurate_pass,team_2_accurate_chipped_pass,team_2_touches,team_2_attempts_conceded_ibox,team_2_saves,team_2_saved_ibox,team_2_ball_recovery,team_2_challenge_lost,team_2_backward_pass,team_2_successful_final_third_passes,t

In [111]:
feature_list_games = ['match_id', 'gameweek_gameweek', 'gameweek_compSeason_label', 'teams_team_1_name', 'teams_team_2_name', 'ground_name', 'team_1_hue']

In [112]:
#Датасет для бейзлайна
df[feature_list_games].to_csv('data/base_line.csv', index=False)

In [113]:
df['month'] = df['kickoff_label'].apply(lambda x: x.split()[2])
df['day_week'] = df['kickoff_label'].apply(lambda x: x.split()[0])
df['hour'] = df['kickoff_label'].apply(lambda x: x.split()[4].split(':')[0])

In [114]:
df_club.head()

,season_id,club_name,club_abbr,club_id,_accurate_back_zone_pass,_accurate_chipped_pass,_accurate_corners_intobox,_accurate_cross,_accurate_cross_nocorner,_accurate_flick_on,_accurate_freekick_cross,_accurate_fwd_zone_pass,_accurate_goal_kicks,_accurate_keeper_sweeper,_accurate_keeper_throws,_accurate_launches,_accurate_layoffs,_accurate_long_balls,_accurate_pass,_accurate_pull_back,_accurate_through_ball,_accurate_throws,_aerial_lost,_aerial_won,_attempted_tackle_foul,_attempts_conceded_ibox,_attempts_conceded_obox,_attempts_ibox,_attempts_obox,_att_assist_openplay,_att_assist_setplay,_att_bx_centre,_att_bx_left,_att_bx_right,_att_cmiss_high,_att_cmiss_left,_att_cmiss_right,_att_corner,_att_fastbreak,_att_goal_high_centre,_att_goal_high_left,_att_goal_high_right,_att_goal_low_centre,_att_goal_low_left,_att_goal_low_right,_att_hd_goal,_att_hd_miss,_att_hd_post,_att_hd_target,_att_hd_total,_att_ibox_blocked,_att_ibox_goal,_att_ibox_miss,_att_ibox_post,_att_ibox_target,_att_lf_goal,_att_lf_target,_att_lf_total,_att_miss_high,_att_miss_high_left,_att_miss_high_right,_att_miss_left,_att_miss_right,_att_obox_blocked,_att_obox_goal,_att_obox_miss,_att_obox_target,_att_obx_centre,_att_one_on_one,_att_openplay,_att_pen_goal,_att_post_left,_att_rf_goal,_att_rf_target,_att_rf_total,_att_setpiece,_att_sv_high_centre,_att_sv_high_left,_att_sv_high_right,_att_sv_low_centre,_att_sv_low_left,_att_sv_low_right,_backward_pass,_ball_recovery,_big_chance_created,_big_chance_missed,_big_chance_scored,_blocked_cross,_blocked_pass,_blocked_scoring_att,_challenge_lost,_clean_sheet,_clearance_off_line,_contentious_decision,_corner_taken,_crosses_18yard,_crosses_18yardplus,_defender_goals,_dispossessed,_diving_save,_draws,_duel_lost,_duel_won,_effective_blocked_cross,_effective_clearance,_effective_head_clearance,_error_lead_to_goal,_error_lead_to_shot,_final_third_entries,_first_half_goals,_fk_foul_lost,_fk_foul_won,_forward_goals,_fouled_final_third,_freekick_cross,_fwd_pass,_goals,_goals_conceded,_goals_conceded_ibox,_goals_conceded_obox,_goals_openplay,_goal_assist,_goal_assist_deadball,_goal_assist_intentional,_goal_assist_openplay,_goal_assist_setplay,_goal_fastbreak,_goal_kicks,_good_high_claim,_hand_ball,_head_clearance,_hit_woodwork,_interception,_interceptions_in_box,_interception_won,_keeper_throws,_last_man_tackle,_leftside_pass,_long_pass_own_to_opp,_long_pass_own_to_opp_success,_lost_corners,_midfielder_goals,_offtarget_att_assist,_ontarget_att_assist,_ontarget_scoring_att,_open_play_pass,_outfielder_block,_overrun,_own_goal_accrued,_passes_left,_passes_right,_penalty_conceded,_penalty_faced,_penalty_won,_pen_area_entries,_pen_goals_conceded,_possession_percentage,_poss_lost_all,_poss_lost_ctrl,_poss_won_att_3rd,_poss_won_def_3rd,_poss_won_mid_3rd,_post_scoring_att,_pts_dropped_winning_pos,_pts_gained_losing_pos,_punches,_put_through,_rightside_pass,_saved_ibox,_saved_obox,_saves,_second_yellow,_shield_ball_oop,_shot_fastbreak,_shot_off_target,_six_yard_block,_subs_goals,_subs_made,_successful_final_third_passes,_successful_open_play_pass,_successful_put_through,_total_att_assist,_total_back_zone_pass,_total_chipped_pass,_total_clearance,_total_contest,_total_corners_intobox,_total_cross,_total_cross_nocorner,_total_fastbreak,_total_final_third_passes,_total_flick_on,_total_fwd_zone_pass,_total_high_claim,_total_keeper_sweeper,_total_launches,_total_layoffs,_total_long_balls,_total_offside,_total_pass,_total_pull_back,_total_red_card,_total_scoring_att,_total_tackle,_total_through_ball,_total_throws,_total_yel_card,_touches,_touches_in_opp_box,_unsuccessful_touch,_wins,_won_contest,_won_corners,_won_tackle,_attendance_count,_attendance_total,_attendance_average,_attendance_highest,_attendance_lowest,season
0,578,Arsenal,ARS,1.0,4351.0,339.0,37.0,100.0,64.0,13.0,7.0,5627.0,78.0,9.0,107.0,55.0,104.0,397.0,9878.0,7.0,21.0,252.0,260.0,249.0,118.0,111.0,61.0,228.0,97.0,199.0,24.0,101.0,38.0,37.0,9.0,11.0,14.0,65.0,7.0,1.0,4.0,4.0,7.0,13.0,7.0,

In [115]:
df_club.drop(labels=['_attendance_count', '_attendance_total', '_attendance_average',	'_attendance_highest', '_attendance_lowest'], axis=1, inplace=True)

In [116]:
list_na = list(df_club.isna().sum()[df_club.isna().sum() > 0].index)

In [117]:
df_club[list_na] = df_club[list_na].fillna(df_club[df_club['season']!=2023][list_na].mean())

In [118]:
cols_mean = list(df_club.columns)[4:-1]
cols_mean = ['club_name'] + cols_mean

In [119]:
df_club_lag = df_club.drop(labels=['season_id', 'club_abbr', 'club_id'], axis=1)
df_club_lag['season'] = df_club_lag['season'] + 1
df_club_lag

,club_name,_accurate_back_zone_pass,_accurate_chipped_pass,_accurate_corners_intobox,_accurate_cross,_accurate_cross_nocorner,_accurate_flick_on,_accurate_freekick_cross,_accurate_fwd_zone_pass,_accurate_goal_kicks,_accurate_keeper_sweeper,_accurate_keeper_throws,_accurate_launches,_accurate_layoffs,_accurate_long_balls,_accurate_pass,_accurate_pull_back,_accurate_through_ball,_accurate_throws,_aerial_lost,_aerial_won,_attempted_tackle_foul,_attempts_conceded_ibox,_attempts_conceded_obox,_attempts_ibox,_attempts_obox,_att_assist_openplay,_att_assist_setplay,_att_bx_centre,_att_bx_left,_att_bx_right,_att_cmiss_high,_att_cmiss_left,_att_cmiss_right,_att_corner,_att_fastbreak,_att_goal_high_centre,_att_goal_high_left,_att_goal_high_right,_att_goal_low_centre,_att_goal_low_left,_att_goal_low_right,_att_hd_goal,_att_hd_miss,_att_hd_post,_att_hd_target,_att_hd_total,_att_ibox_blocked,_att_ibox_goal,_att_ibox_miss,_att_ibox_post,_att_ibox_target,_att_lf_goal,_att_lf_target,_att_lf_total,_att_miss_high,_att_miss_high_left,_att_miss_high_right,_att_miss_left,_att_miss_right,_att_obox_blocked,_att_obox_goal,_att_obox_miss,_att_obox_target,_att_obx_centre,_att_one_on_one,_att_openplay,_att_pen_goal,_att_post_left,_att_rf_goal,_att_rf_target,_att_rf_total,_att_setpiece,_att_sv_high_centre,_att_sv_high_left,_att_sv_high_right,_att_sv_low_centre,_att_sv_low_left,_att_sv_low_right,_backward_pass,_ball_recovery,_big_chance_created,_big_chance_missed,_big_chance_scored,_blocked_cross,_blocked_pass,_blocked_scoring_att,_challenge_lost,_clean_sheet,_clearance_off_line,_contentious_decision,_corner_taken,_crosses_18yard,_crosses_18yardplus,_defender_goals,_dispossessed,_diving_save,_draws,_duel_lost,_duel_won,_effective_blocked_cross,_effective_clearance,_effective_head_clearance,_error_lead_to_goal,_error_lead_to_shot,_final_third_entries,_first_half_goals,_fk_foul_lost,_fk_foul_won,_forward_goals,_fouled_final_third,_freekick_cross,_fwd_pass,_goals,_goals_conceded,_goals_conceded_ibox,_goals_conceded_obox,_goals_openplay,_goal_assist,_goal_assist_deadball,_goal_assist_intentional,_goal_assist_openplay,_goal_assist_setplay,_goal_fastbreak,_goal_kicks,_good_high_claim,_hand_ball,_head_clearance,_hit_woodwork,_interception,_interceptions_in_box,_interception_won,_keeper_throws,_last_man_tackle,_leftside_pass,_long_pass_own_to_opp,_long_pass_own_to_opp_success,_lost_corners,_midfielder_goals,_offtarget_att_assist,_ontarget_att_assist,_ontarget_scoring_att,_open_play_pass,_outfielder_block,_overrun,_own_goal_accrued,_passes_left,_passes_right,_penalty_conceded,_penalty_faced,_penalty_won,_pen_area_entries,_pen_goals_conceded,_possession_percentage,_poss_lost_all,_poss_lost_ctrl,_poss_won_att_3rd,_poss_won_def_3rd,_poss_won_mid_3rd,_post_scoring_att,_pts_dropped_winning_pos,_pts_gained_losing_pos,_punches,_put_through,_rightside_pass,_saved_ibox,_saved_obox,_saves,_second_yellow,_shield_ball_oop,_shot_fastbreak,_shot_off_target,_six_yard_block,_subs_goals,_subs_made,_successful_final_third_passes,_successful_open_play_pass,_successful_put_through,_total_att_assist,_total_back_zone_pass,_total_chipped_pass,_total_clearance,_total_contest,_total_corners_intobox,_total_cross,_total_cross_nocorner,_total_fastbreak,_total_final_third_passes,_total_flick_on,_total_fwd_zone_pass,_total_high_claim,_total_keeper_sweeper,_total_launches,_total_layoffs,_total_long_balls,_total_offside,_total_pass,_total_pull_back,_total_red_card,_total_scoring_att,_total_tackle,_total_through_ball,_total_throws,_total_yel_card,_touches,_touches_in_opp_box,_unsuccessful_touch,_wins,_won_contest,_won_corners,_won_tackle,season
0,Arsenal,4351.0,339.0,37.0,100.0,64.0,13.0,7.0,5627.0,78.0,9.0,107.0,55.0,104.0,397.0,9878.0,7.0,21.0,252.0,260.0,249.0,118.0,111.0,61.0,228.0,97.0,199.0,24.0,101.0,38.0,37.0,9.0,11.0,14.0,65.0,7.0,1.000000,4.0,4.0,7.0,13.0,7.0,9.0,27.0,1.000000,7.0,52.0,58.0,33.0,74.0,5.0,59.0,12.0,33.0,126.0,25.0,12.0,7.0,32.0,28.0,48.0,4.000000,30.0,15.0,97.0,4.0,241.0,

In [120]:
df_num_mean = df_club[df_club['season']!=2023][cols_mean].groupby('club_name', as_index=False).mean()

In [121]:
df_num_mean.head()

,club_name,_accurate_back_zone_pass,_accurate_chipped_pass,_accurate_corners_intobox,_accurate_cross,_accurate_cross_nocorner,_accurate_flick_on,_accurate_freekick_cross,_accurate_fwd_zone_pass,_accurate_goal_kicks,_accurate_keeper_sweeper,_accurate_keeper_throws,_accurate_launches,_accurate_layoffs,_accurate_long_balls,_accurate_pass,_accurate_pull_back,_accurate_through_ball,_accurate_throws,_aerial_lost,_aerial_won,_attempted_tackle_foul,_attempts_conceded_ibox,_attempts_conceded_obox,_attempts_ibox,_attempts_obox,_att_assist_openplay,_att_assist_setplay,_att_bx_centre,_att_bx_left,_att_bx_right,_att_cmiss_high,_att_cmiss_left,_att_cmiss_right,_att_corner,_att_fastbreak,_att_goal_high_centre,_att_goal_high_left,_att_goal_high_right,_att_goal_low_centre,_att_goal_low_left,_att_goal_low_right,_att_hd_goal,_att_hd_miss,_att_hd_post,_att_hd_target,_att_hd_total,_att_ibox_blocked,_att_ibox_goal,_att_ibox_miss,_att_ibox_post,_att_ibox_target,_att_lf_goal,_att_lf_target,_att_lf_total,_att_miss_high,_att_miss_high_left,_att_miss_high_right,_att_miss_left,_att_miss_right,_att_obox_blocked,_att_obox_goal,_att_obox_miss,_att_obox_target,_att_obx_centre,_att_one_on_one,_att_openplay,_att_pen_goal,_att_post_left,_att_rf_goal,_att_rf_target,_att_rf_total,_att_setpiece,_att_sv_high_centre,_att_sv_high_left,_att_sv_high_right,_att_sv_low_centre,_att_sv_low_left,_att_sv_low_right,_backward_pass,_ball_recovery,_big_chance_created,_big_chance_missed,_big_chance_scored,_blocked_cross,_blocked_pass,_blocked_scoring_att,_challenge_lost,_clean_sheet,_clearance_off_line,_contentious_decision,_corner_taken,_crosses_18yard,_crosses_18yardplus,_defender_goals,_dispossessed,_diving_save,_draws,_duel_lost,_duel_won,_effective_blocked_cross,_effective_clearance,_effective_head_clearance,_error_lead_to_goal,_error_lead_to_shot,_final_third_entries,_first_half_goals,_fk_foul_lost,_fk_foul_won,_forward_goals,_fouled_final_third,_freekick_cross,_fwd_pass,_goals,_goals_conceded,_goals_conceded_ibox,_goals_conceded_obox,_goals_openplay,_goal_assist,_goal_assist_deadball,_goal_assist_intentional,_goal_assist_openplay,_goal_assist_setplay,_goal_fastbreak,_goal_kicks,_good_high_claim,_hand_ball,_head_clearance,_hit_woodwork,_interception,_interceptions_in_box,_interception_won,_keeper_throws,_last_man_tackle,_leftside_pass,_long_pass_own_to_opp,_long_pass_own_to_opp_success,_lost_corners,_midfielder_goals,_offtarget_att_assist,_ontarget_att_assist,_ontarget_scoring_att,_open_play_pass,_outfielder_block,_overrun,_own_goal_accrued,_passes_left,_passes_right,_penalty_conceded,_penalty_faced,_penalty_won,_pen_area_entries,_pen_goals_conceded,_possession_percentage,_poss_lost_all,_poss_lost_ctrl,_poss_won_att_3rd,_poss_won_def_3rd,_poss_won_mid_3rd,_post_scoring_att,_pts_dropped_winning_pos,_pts_gained_losing_pos,_punches,_put_through,_rightside_pass,_saved_ibox,_saved_obox,_saves,_second_yellow,_shield_ball_oop,_shot_fastbreak,_shot_off_target,_six_yard_block,_subs_goals,_subs_made,_successful_final_third_passes,_successful_open_play_pass,_successful_put_through,_total_att_assist,_total_back_zone_pass,_total_chipped_pass,_total_clearance,_total_contest,_total_corners_intobox,_total_cross,_total_cross_nocorner,_total_fastbreak,_total_final_third_passes,_total_flick_on,_total_fwd_zone_pass,_total_high_claim,_total_keeper_sweeper,_total_launches,_total_layoffs,_total_long_balls,_total_offside,_total_pass,_total_pull_back,_total_red_card,_total_scoring_att,_total_tackle,_total_through_ball,_total_throws,_total_yel_card,_touches,_touches_in_opp_box,_unsuccessful_touch,_wins,_won_contest,_won_corners,_won_tackle
0,Arsenal,8158.50,626.75,45.25,147.25,104.75,28.00,10.75,8341.75,152.00,16.25,194.5,82.75,181.75,862.50,16353.00,14.75,25.0,588.00,586.00,497.00,206.00,278.75,156.50,340.00,171.75,295.75,37.50,181.50,45.75,50.00,9.25,13.5,22.25,84.00,16.00,4.25,6.75,4.50,12.75,20.50,18.75,8.25,35.00,2.000000,15.25,65.50,84.75,58.50,111.25,9.00,79.00,26.00,43.75,210.50,38.0,20.75,2

In [122]:
df_lag = df_club_lag[df_club_lag['season']==2023]
df_lag.to_csv('data/df_lag.csv', index=False)

In [123]:
df_num_mean.to_csv('data/club_mean.csv', index=False)


In [124]:
df_1 = df[feature_list_games].merge(df_num_mean, left_on='teams_team_1_name', right_on='club_name')
df_1 = df_1.merge(df_num_mean, left_on='teams_team_2_name', right_on='club_name', suffixes=('_team_1', '_team_2'))
df_1.drop(['club_name_team_1', 'club_name_team_2'], axis=1, inplace=True)
df_1

,match_id,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue,_accurate_back_zone_pass_team_1,_accurate_chipped_pass_team_1,_accurate_corners_intobox_team_1,_accurate_cross_team_1,_accurate_cross_nocorner_team_1,_accurate_flick_on_team_1,_accurate_freekick_cross_team_1,_accurate_fwd_zone_pass_team_1,_accurate_goal_kicks_team_1,_accurate_keeper_sweeper_team_1,_accurate_keeper_throws_team_1,_accurate_launches_team_1,_accurate_layoffs_team_1,_accurate_long_balls_team_1,_accurate_pass_team_1,_accurate_pull_back_team_1,_accurate_through_ball_team_1,_accurate_throws_team_1,_aerial_lost_team_1,_aerial_won_team_1,_attempted_tackle_foul_team_1,_attempts_conceded_ibox_team_1,_attempts_conceded_obox_team_1,_attempts_ibox_team_1,_attempts_obox_team_1,_att_assist_openplay_team_1,_att_assist_setplay_team_1,_att_bx_centre_team_1,_att_bx_left_team_1,_att_bx_right_team_1,_att_cmiss_high_team_1,_att_cmiss_left_team_1,_att_cmiss_right_team_1,_att_corner_team_1,_att_fastbreak_team_1,_att_goal_high_centre_team_1,_att_goal_high_left_team_1,_att_goal_high_right_team_1,_att_goal_low_centre_team_1,_att_goal_low_left_team_1,_att_goal_low_right_team_1,_att_hd_goal_team_1,_att_hd_miss_team_1,_att_hd_post_team_1,_att_hd_target_team_1,_att_hd_total_team_1,_att_ibox_blocked_team_1,_att_ibox_goal_team_1,_att_ibox_miss_team_1,_att_ibox_post_team_1,_att_ibox_target_team_1,_att_lf_goal_team_1,_att_lf_target_team_1,_att_lf_total_team_1,_att_miss_high_team_1,_att_miss_high_left_team_1,_att_miss_high_right_team_1,_att_miss_left_team_1,_att_miss_right_team_1,_att_obox_blocked_team_1,_att_obox_goal_team_1,_att_obox_miss_team_1,_att_obox_target_team_1,_att_obx_centre_team_1,_att_one_on_one_team_1,_att_openplay_team_1,_att_pen_goal_team_1,_att_post_left_team_1,_att_rf_goal_team_1,_att_rf_target_team_1,_att_rf_total_team_1,_att_setpiece_team_1,_att_sv_high_centre_team_1,_att_sv_high_left_team_1,_att_sv_high_right_team_1,_att_sv_low_centre_team_1,_att_sv_low_left_team_1,_att_sv_low_right_team_1,_backward_pass_team_1,_ball_recovery_team_1,_big_chance_created_team_1,_big_chance_missed_team_1,_big_chance_scored_team_1,_blocked_cross_team_1,_blocked_pass_team_1,_blocked_scoring_att_team_1,_challenge_lost_team_1,_clean_sheet_team_1,_clearance_off_line_team_1,_contentious_decision_team_1,_corner_taken_team_1,_crosses_18yard_team_1,_crosses_18yardplus_team_1,_defender_goals_team_1,_dispossessed_team_1,_diving_save_team_1,_draws_team_1,_duel_lost_team_1,_duel_won_team_1,_effective_blocked_cross_team_1,_effective_clearance_team_1,_effective_head_clearance_team_1,_error_lead_to_goal_team_1,_error_lead_to_shot_team_1,_final_third_entries_team_1,_first_half_goals_team_1,_fk_foul_lost_team_1,_fk_foul_won_team_1,_forward_goals_team_1,_fouled_final_third_team_1,_freekick_cross_team_1,_fwd_pass_team_1,_goals_team_1,_goals_conceded_team_1,_goals_conceded_ibox_team_1,_goals_conceded_obox_team_1,_goals_openplay_team_1,_goal_assist_team_1,_goal_assist_deadball_team_1,_goal_assist_intentional_team_1,_goal_assist_openplay_team_1,_goal_assist_setplay_team_1,_goal_fastbreak_team_1,_goal_kicks_team_1,_good_high_claim_team_1,_hand_ball_team_1,_head_clearance_team_1,_hit_woodwork_team_1,_interception_team_1,_interceptions_in_box_team_1,_interception_won_team_1,_keeper_throws_team_1,_last_man_tackle_team_1,_leftside_pass_team_1,_long_pass_own_to_opp_team_1,_long_pass_own_to_opp_success_team_1,_lost_corners_team_1,_midfielder_goals_team_1,_offtarget_att_assist_team_1,_ontarget_att_assist_team_1,_ontarget_scoring_att_team_1,_open_play_pass_team_1,_outfielder_block_team_1,_overrun_team_1,_own_goal_accrued_team_1,_passes_left_team_1,_passes_right_team_1,_penalty_conceded_team_1,_penalty_faced_team_1,_penalty_won_team_1,_pen_area_entries_team_1,_pen_goals_conceded_team_1,_possession_percentage_team_1,_poss_lost_all_team_1,_poss_lost_ctrl_team_1,_poss_won_att_3rd_team_1,_poss_won_def_3rd_team_1,_poss_won_mid_3rd_team_1,_post_scoring_att_team_1,_pts_drop

In [125]:
df_res = pd.merge(df_1, df_club_lag, left_on=['teams_team_1_name', 'gameweek_compSeason_label'], right_on=['club_name', 'season'], how='left')
df_res = pd.merge(df_res, df_club_lag, left_on=['teams_team_2_name', 'gameweek_compSeason_label'], right_on=['club_name', 'season'], how='left', suffixes=('_lag_team1', '_lag_team2'))

In [126]:
df_res.sort_values(by=['match_id'], ascending=False , inplace=True)
df_res.drop('match_id', axis=1, inplace=True,)
df_res

gameweek_gameweek  gameweek_compSeason_label        teams_team_1_name  \
305                  20                       2023  Wolverhampton Wanderers   
1132                 20                       2023        Tottenham Hotspur   
18                   20                       2023        Nottingham Forest   
450                  20                       2023          Manchester City   
359                  20                       2023                Liverpool   
...                 ...                        ...                      ...   
153                   1                       2019           Leicester City   
300                   1                       2019           Crystal Palace   
907                   1                       2019                  Burnley   
468                   1                       2019              Bournemouth   
1397                  1                       2019                Liverpool   

            teams_team_2_name                ground_name  team_1_hue  \
305                   Everton           Molineux Stadium           1   
1132              Bournemouth  Tottenham Hotspur Stadium           1   
18          Manchester United            The City Ground           1   
450          Sheffield United             Etihad Stadium           1   
359          Newcastle United                    Anfield           1   
...                       ...                        ...         ...   
153   Wolverhampton Wanderers         King Power Stadium           2   
300                   Everton              Selhurst Park           2   
907               Southampton                  Turf Moor           1   
468          Sheffield United           Vitality Stadium           2   
1397             Norwich City                    Anfield           1   

      _accurate_back_zone_pass_team_1  _accurate_chipped_pass_team_1  \
305                           7582.25                     832.750000   
1132                          8308.50                     685.500000   
18                            5016.00                     601.000000   
450                          10041.25                     918.750000   
359                           9401.25                     967.750000   
...                               ...                            ...   
153                           8310.50                     643.500000   
300                           6535.25                     618.500000   
907                           4121.00                     636.666667   
468                           5986.50                     578.000000   
1397                          9401.25                     967.750000   

      _accurate_corners_intobox_team_1  _accurate_cross_team_1  \
305                          43.750000              160.250000   
1132                         41.750000              140.500000   
18                           29.000000              104.000000   
450                          65.750000              168.500000   
359                          64.250000              195.500000   
...                                ...                     ...   
153                          38.250000              124.500000   
300                          46.000000              141.000000   
907                          46.333333              165.333333   
468                          43.000000              132.500000   
1397                         64.250000              195.500000   

      _accurate_cross_nocorner_team_1  _accurate_flick_on_team_1  \
305                        118.250000                      52.75   
1132                       100.500000                      47.00   
18                          76.000000                      72.00   
450                        112.000000                      27.50   
359                        132.000000                      37.00   
...                               ...                        ...   
153                         90.000000                     

In [127]:
df_res.drop(['season_lag_team2', 'season_lag_team1', 'club_name_lag_team1', 'club_name_lag_team2'], axis=1, inplace=True)

In [128]:
df_res.shape

(1528, 846)

In [129]:
df_res.to_csv('data/df_res.csv', index=False)

In [130]:
df_1.to_csv('data/df_1.csv', index=False)

Удалим признаки с высокой корреляцией

In [131]:
df_1.iloc[:, 6:].corr()

,team_1_hue,_accurate_back_zone_pass_team_1,_accurate_chipped_pass_team_1,_accurate_corners_intobox_team_1,_accurate_cross_team_1,_accurate_cross_nocorner_team_1,_accurate_flick_on_team_1,_accurate_freekick_cross_team_1,_accurate_fwd_zone_pass_team_1,_accurate_goal_kicks_team_1,_accurate_keeper_sweeper_team_1,_accurate_keeper_throws_team_1,_accurate_launches_team_1,_accurate_layoffs_team_1,_accurate_long_balls_team_1,_accurate_pass_team_1,_accurate_pull_back_team_1,_accurate_through_ball_team_1,_accurate_throws_team_1,_aerial_lost_team_1,_aerial_won_team_1,_attempted_tackle_foul_team_1,_attempts_conceded_ibox_team_1,_attempts_conceded_obox_team_1,_attempts_ibox_team_1,_attempts_obox_team_1,_att_assist_openplay_team_1,_att_assist_setplay_team_1,_att_bx_centre_team_1,_att_bx_left_team_1,_att_bx_right_team_1,_att_cmiss_high_team_1,_att_cmiss_left_team_1,_att_cmiss_right_team_1,_att_corner_team_1,_att_fastbreak_team_1,_att_goal_high_centre_team_1,_att_goal_high_left_team_1,_att_goal_high_right_team_1,_att_goal_low_centre_team_1,_att_goal_low_left_team_1,_att_goal_low_right_team_1,_att_hd_goal_team_1,_att_hd_miss_team_1,_att_hd_post_team_1,_att_hd_target_team_1,_att_hd_total_team_1,_att_ibox_blocked_team_1,_att_ibox_goal_team_1,_att_ibox_miss_team_1,_att_ibox_post_team_1,_att_ibox_target_team_1,_att_lf_goal_team_1,_att_lf_target_team_1,_att_lf_total_team_1,_att_miss_high_team_1,_att_miss_high_left_team_1,_att_miss_high_right_team_1,_att_miss_left_team_1,_att_miss_right_team_1,_att_obox_blocked_team_1,_att_obox_goal_team_1,_att_obox_miss_team_1,_att_obox_target_team_1,_att_obx_centre_team_1,_att_one_on_one_team_1,_att_openplay_team_1,_att_pen_goal_team_1,_att_post_left_team_1,_att_rf_goal_team_1,_att_rf_target_team_1,_att_rf_total_team_1,_att_setpiece_team_1,_att_sv_high_centre_team_1,_att_sv_high_left_team_1,_att_sv_high_right_team_1,_att_sv_low_centre_team_1,_att_sv_low_left_team_1,_att_sv_low_right_team_1,_backward_pass_team_1,_ball_recovery_team_1,_big_chance_created_team_1,_big_chance_missed_team_1,_big_chance_scored_team_1,_blocked_cross_team_1,_blocked_pass_team_1,_blocked_scoring_att_team_1,_challenge_lost_team_1,_clean_sheet_team_1,_clearance_off_line_team_1,_contentious_decision_team_1,_corner_taken_team_1,_crosses_18yard_team_1,_crosses_18yardplus_team_1,_defender_goals_team_1,_dispossessed_team_1,_diving_save_team_1,_draws_team_1,_duel_lost_team_1,_duel_won_team_1,_effective_blocked_cross_team_1,_effective_clearance_team_1,_effective_head_clearance_team_1,_error_lead_to_goal_team_1,_error_lead_to_shot_team_1,_final_third_entries_team_1,_first_half_goals_team_1,_fk_foul_lost_team_1,_fk_foul_won_team_1,_forward_goals_team_1,_fouled_final_third_team_1,_freekick_cross_team_1,_fwd_pass_team_1,_goals_team_1,_goals_conceded_team_1,_goals_conceded_ibox_team_1,_goals_conceded_obox_team_1,_goals_openplay_team_1,_goal_assist_team_1,_goal_assist_deadball_team_1,_goal_assist_intentional_team_1,_goal_assist_openplay_team_1,_goal_assist_setplay_team_1,_goal_fastbreak_team_1,_goal_kicks_team_1,_good_high_claim_team_1,_hand_ball_team_1,_head_clearance_team_1,_hit_woodwork_team_1,_interception_team_1,_interceptions_in_box_team_1,_interception_won_team_1,_keeper_throws_team_1,_last_man_tackle_team_1,_leftside_pass_team_1,_long_pass_own_to_opp_team_1,_long_pass_own_to_opp_success_team_1,_lost_corners_team_1,_midfielder_goals_team_1,_offtarget_att_assist_team_1,_ontarget_att_assist_team_1,_ontarget_scoring_att_team_1,_open_play_pass_team_1,_outfielder_block_team_1,_overrun_team_1,_own_goal_accrued_team_1,_passes_left_team_1,_passes_right_team_1,_penalty_conceded_team_1,_penalty_faced_team_1,_penalty_won_team_1,_pen_area_entries_team_1,_pen_goals_conceded_team_1,_possession_percentage_team_1,_poss_lost_all_team_1,_poss_lost_ctrl_team_1,_poss_won_att_3rd_team_1,_poss_won_def_3rd_team_1,_poss_won_mid_3rd_team_1,_post_scoring_att_team_1,_pts_dropped_winning_pos_team_1,_pts_gained_losing_pos_team_1,_punches_team_1,_put_through_team_1,_rightside_p

In [132]:
def rm_high_corr_feat(df, thr, exc_col):
    features_to_remove = set()

    corr_matrix = df.iloc[:, exc_col:].corr()

    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > thr:
                colname = corr_matrix.columns[i]
                features_to_remove.add(colname)

    # Удаляем признаки из исходного датафрейма
    df = df.drop(columns=features_to_remove)

    # Рекурсивный вызов функции, если были удалены признаки
    if len(features_to_remove) > 0:
        return rm_high_corr_feat(df, thr, exc_col)
    else:
        return df

In [133]:
df_rm_hcorr = rm_high_corr_feat(df_1, 0.8, 6)
df_rm_hcorr.shape

(1528, 153)

In [134]:
df_rm_hcorr.head()

,match_id,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue,_accurate_back_zone_pass_team_1,_accurate_chipped_pass_team_1,_accurate_corners_intobox_team_1,_accurate_cross_team_1,_accurate_freekick_cross_team_1,_accurate_goal_kicks_team_1,_accurate_keeper_sweeper_team_1,_accurate_keeper_throws_team_1,_accurate_long_balls_team_1,_accurate_pull_back_team_1,_accurate_through_ball_team_1,_accurate_throws_team_1,_attempted_tackle_foul_team_1,_att_cmiss_high_team_1,_att_fastbreak_team_1,_att_goal_high_centre_team_1,_att_hd_goal_team_1,_att_hd_post_team_1,_att_hd_target_team_1,_att_ibox_post_team_1,_att_miss_high_left_team_1,_att_miss_high_right_team_1,_att_pen_goal_team_1,_att_post_left_team_1,_att_sv_high_centre_team_1,_att_sv_high_left_team_1,_att_sv_high_right_team_1,_ball_recovery_team_1,_blocked_pass_team_1,_challenge_lost_team_1,_clearance_off_line_team_1,_contentious_decision_team_1,_crosses_18yard_team_1,_crosses_18yardplus_team_1,_defender_goals_team_1,_dispossessed_team_1,_draws_team_1,_duel_lost_team_1,_duel_won_team_1,_error_lead_to_goal_team_1,_error_lead_to_shot_team_1,_fk_foul_won_team_1,_fouled_final_third_team_1,_goals_conceded_obox_team_1,_goal_assist_deadball_team_1,_goal_assist_setplay_team_1,_goal_fastbreak_team_1,_good_high_claim_team_1,_hand_ball_team_1,_interception_team_1,_last_man_tackle_team_1,_long_pass_own_to_opp_team_1,_midfielder_goals_team_1,_overrun_team_1,_own_goal_accrued_team_1,_penalty_conceded_team_1,_poss_won_def_3rd_team_1,_pts_dropped_winning_pos_team_1,_pts_gained_losing_pos_team_1,_punches_team_1,_put_through_team_1,_second_yellow_team_1,_shield_ball_oop_team_1,_six_yard_block_team_1,_subs_goals_team_1,_subs_made_team_1,_successful_put_through_team_1,_total_chipped_pass_team_1,_total_contest_team_1,_total_offside_team_1,_total_red_card_team_1,_total_yel_card_team_1,_unsuccessful_touch_team_1,_accurate_back_zone_pass_team_2,_accurate_chipped_pass_team_2,_accurate_corners_intobox_team_2,_accurate_cross_team_2,_accurate_freekick_cross_team_2,_accurate_goal_kicks_team_2,_accurate_keeper_sweeper_team_2,_accurate_keeper_throws_team_2,_accurate_long_balls_team_2,_accurate_pull_back_team_2,_accurate_through_ball_team_2,_accurate_throws_team_2,_attempted_tackle_foul_team_2,_att_cmiss_high_team_2,_att_fastbreak_team_2,_att_goal_high_centre_team_2,_att_hd_goal_team_2,_att_hd_post_team_2,_att_hd_target_team_2,_att_ibox_post_team_2,_att_miss_high_left_team_2,_att_miss_high_right_team_2,_att_pen_goal_team_2,_att_post_left_team_2,_att_sv_high_centre_team_2,_att_sv_high_left_team_2,_att_sv_high_right_team_2,_ball_recovery_team_2,_blocked_pass_team_2,_challenge_lost_team_2,_clearance_off_line_team_2,_contentious_decision_team_2,_crosses_18yard_team_2,_crosses_18yardplus_team_2,_defender_goals_team_2,_dispossessed_team_2,_draws_team_2,_duel_lost_team_2,_duel_won_team_2,_error_lead_to_goal_team_2,_error_lead_to_shot_team_2,_fk_foul_won_team_2,_fouled_final_third_team_2,_goals_conceded_obox_team_2,_goal_assist_deadball_team_2,_goal_assist_setplay_team_2,_goal_fastbreak_team_2,_good_high_claim_team_2,_hand_ball_team_2,_interception_team_2,_last_man_tackle_team_2,_long_pass_own_to_opp_team_2,_midfielder_goals_team_2,_overrun_team_2,_own_goal_accrued_team_2,_penalty_conceded_team_2,_poss_won_def_3rd_team_2,_pts_dropped_winning_pos_team_2,_pts_gained_losing_pos_team_2,_punches_team_2,_put_through_team_2,_second_yellow_team_2,_shield_ball_oop_team_2,_six_yard_block_team_2,_subs_goals_team_2,_subs_made_team_2,_successful_put_through_team_2,_total_chipped_pass_team_2,_total_contest_team_2,_total_offside_team_2,_total_red_card_team_2,_total_yel_card_team_2,_unsuccessful_touch_team_2
0,93499,18,2023,West Ham United,Manchester United,London Stadium,1,5829.5,648.75,50.75,169.5,13.75,185.25,7.5,155.5,964.5,9.75,12.5,622.75,202.75,11.0,21.5,3.75,9.5,3.75,25.25,8.25,13.0,19.5,4.0,4.5,14.5,7.5,5.75,2284.0,563.5,350.75,3.0,15.0,368.5,182.75,7.5,322.5,8.0,1941.25,1935.75,4.75

In [135]:
df_rm_hcorr.to_csv('data/df_rm_hcorr.csv', index=False)

## FOR TS

In [136]:
df.head()

,match_id,gameweek_id,gameweek_compSeason_label,gameweek_compSeason_id,gameweek_gameweek,kickoff_millis,kickoff_label,provisionalKickoff_millis,provisionalKickoff_label,teams_team_1_name,teams_team_1_club_abbr,teams_team_1_club_id,teams_score_1,teams_team_2_name,teams_team_2_club_abbr,teams_team_2_club_id,teams_score_2,ground_name,ground_city,ground_id,outcome,attendance,clock_secs,clock_label,behindClosedDoors,id,team_1_formation_used,team_1_accurate_back_zone_pass,team_1_total_back_zone_pass,team_1_accurate_pass,team_1_touches,team_1_total_pass,team_1_backward_pass,team_1_successful_open_play_pass,team_1_open_play_pass,team_1_possession_percentage,team_1_leftside_pass,team_1_accurate_fwd_zone_pass,team_1_rightside_pass,team_1_total_long_balls,team_1_total_fwd_zone_pass,team_1_touches_in_opp_box,team_1_final_third_entries,team_1_crosses_18yard,team_1_successful_final_third_passes,team_1_accurate_cross,team_1_long_pass_own_to_opp,team_1_fwd_pass,team_1_total_cross_nocorner,team_1_total_cross,team_1_total_final_third_passes,team_1_long_pass_own_to_opp_success,team_1_poss_lost_all,team_1_poss_lost_ctrl,team_1_pen_area_entries,team_1_passes_left,team_1_accurate_long_balls,team_1_crosses_18yardplus,team_1_accurate_cross_nocorner,team_1_attempts_ibox,team_1_total_scoring_att,team_1_att_openplay,team_1_att_lf_target,team_1_att_bx_centre,team_1_ontarget_scoring_att,team_1_ball_recovery,team_1_att_lf_total,team_1_poss_won_att_3rd,team_1_att_ibox_target,team_1_ontarget_att_assist,team_1_blocked_pass,team_1_total_att_assist,team_1_duel_won,team_1_won_contest,team_1_att_sv_low_centre,team_1_successful_put_through,team_1_total_contest,team_1_att_assist_openplay,team_1_put_through,team_1_poss_won_mid_3rd,team_1_total_tackle,team_1_interception,team_1_interception_won,team_1_won_tackle,team_1_total_offside,team_1_duel_lost,team_1_accurate_layoffs,team_1_passes_right,team_1_total_chipped_pass,team_1_total_layoffs,team_1_accurate_chipped_pass,team_1_aerial_lost,team_1_effective_clearance,team_1_lost_corners,team_1_total_launches,team_1_total_clearance,team_1_attempts_conceded_ibox,team_1_fk_foul_won,team_1_poss_won_def_3rd,team_1_saved_ibox,team_1_saves,team_1_diving_save,team_1_challenge_lost,team_1_dispossessed,team_1_unsuccessful_touch,team_1_goal_kicks,team_1_accurate_goal_kicks,team_1_attempts_conceded_obox,team_1_head_clearance,team_1_effective_head_clearance,team_1_fk_foul_lost,team_1_attempted_tackle_foul,team_1_aerial_won,team_1_goals_conceded_ibox,team_1_goals_conceded,team_1_total_yel_card,team_1_shot_off_target,team_1_total_throws,team_1_att_rf_total,team_1_accurate_throws,team_1_att_ibox_miss,team_1_offtarget_att_assist,team_1_saved_obox,team_1_outfielder_block,team_1_att_rf_target,team_1_corner_taken,team_1_won_corners,team_1_accurate_corners_intobox,team_1_att_assist_setplay,team_1_att_miss_right,team_1_att_corner,team_1_total_corners_intobox,team_1_att_hd_total,team_1_att_hd_miss,team_1_effective_blocked_cross,team_1_blocked_cross,team_1_subs_made,team_1_accurate_launches,team_1_fouled_final_third,team_1_att_obx_centre,team_1_attempts_obox,team_1_att_obox_target,team_1_keeper_throws,team_1_accurate_keeper_throws,team_1_total_through_ball,team_1_big_chance_missed,team_1_big_chance_created,team_1_att_miss_left,team_1_blocked_scoring_att,team_1_att_ibox_blocked,team_1_total_flick_on,team_1_losses,team_2_formation_used,team_2_open_play_pass,team_2_leftside_pass,team_2_accurate_goal_kicks,team_2_possession_percentage,team_2_poss_lost_all,team_2_goal_kicks,team_2_poss_lost_ctrl,team_2_total_pass,team_2_total_long_balls,team_2_total_back_zone_pass,team_2_total_chipped_pass,team_2_successful_open_play_pass,team_2_accurate_long_balls,team_2_accurate_back_zone_pass,team_2_rightside_pass,team_2_unsuccessful_touch,team_2_accurate_pass,team_2_accurate_chipped_pass,team_2_touches,team_2_attempts_conceded_ibox,team_2_saves,team_2_saved_ibox,team_2_ball_recovery,team_2_challenge_lost,team_2_backward_pass,team_2_successful_final_third_passes,t

In [137]:
col_ts_baseline = ['provisionalKickoff_label', 'gameweek_compSeason_label', 'gameweek_gameweek', 'teams_team_1_name', 'teams_team_2_name', 'teams_score_1', 'teams_score_2']

#col_ts_baseline = ['match_id', 'gameweek_gameweek', 'month', 'day_week', 'gameweek_compSeason_label', 'teams_team_1_name', 'teams_team_2_name', 'team_1_hue']

Будем генерировать признак голы 1 команды / голы второй команды, что бы не было деления на ноль прибавляем по голу каждой команде

In [138]:
df_ts_bl = df[col_ts_baseline]

#Приводим дату к норм виду
df_ts_bl.loc[:, 'provisionalKickoff_label'] = df_ts_bl.loc[:, 'provisionalKickoff_label'].apply(lambda x: ' '.join(x.split()[:-1]))
df_ts_bl.loc[:, 'provisionalKickoff_label'] = pd.to_datetime(df_ts_bl.loc[:, 'provisionalKickoff_label'])
df_ts_bl.loc[:, 'month'] = df_ts_bl['provisionalKickoff_label'].dt.month
df_ts_bl = df_ts_bl.sort_values(by='provisionalKickoff_label')
df_ts_bl.drop('provisionalKickoff_label', axis=1, inplace=True)

colum = {
    'gameweek_compSeason_label' : 'season',
    'gameweek_gameweek': 'week',
    'teams_team_1_name': 'team_1',
    'teams_team_2_name': 'team_2',    
}
df_ts_bl = df_ts_bl.rename(colum, axis=1)
df_ts_bl.head()

/tmp/ipykernel_152906/2452901962.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ts_bl.loc[:, 'provisionalKickoff_label'] = df_ts_bl.loc[:, 'provisionalKickoff_label'].apply(lambda x: ' '.join(x.split()[:-1]))
/tmp/ipykernel_152906/2452901962.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ts_bl.loc[:, 'provisionalKickoff_label'] = pd.to_datetime(df_ts_bl.loc[:, 'provisionalKickoff_label'])
/tmp/ipykernel_152906/2452901962.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals

,season,week,team_1,team_2,teams_score_1,teams_score_2,month
1717,2019,1,Liverpool,Norwich City,4,1,8
1716,2019,1,West Ham United,Manchester City,0,5,8
1714,2019,1,Crystal Palace,Everton,0,0,8
1712,2019,1,Bournemouth,Sheffield United,1,1,8
1715,2019,1,Watford,Brighton & Hove Albion,0,3,8


In [139]:
def get_df_ts(df, team='Aston Villa'):

    df_t = df[(df['team_1']==team) | (df['team_2']==team)]

    df_t.loc[:, 'home'] = np.where(df_t.loc[:, 'team_1']==team, 1, 0)

    df_t.loc[:, 'score'] = (df_t.loc[:, 'teams_score_1'] + 1) /(df_t.loc[:, 'teams_score_2'] + 1) #Коэффициент забиты голов первой команды ко второй
    # Если команда гостевая (тим2) то переворачиваем скор
    df_t.loc[:, 'score'] = np.where(df_t.loc[:, 'team_1']==team, df_t.loc[:, 'score'], 1 / df_t.loc[:, 'score'])

    df_t = df_t.drop(['team_1', 'team_2', 'teams_score_1', 'teams_score_2'], axis=1)

    return df_t

In [140]:
get_df_ts(df_ts_bl, team='Aston Villa')

/tmp/ipykernel_152906/3638366935.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t.loc[:, 'home'] = np.where(df_t.loc[:, 'team_1']==team, 1, 0)
/tmp/ipykernel_152906/3638366935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t.loc[:, 'score'] = (df_t.loc[:, 'teams_score_1'] + 1) /(df_t.loc[:, 'teams_score_2'] + 1) #Коэффициент забиты голов первой команды ко второй
/tmp/ipykernel_152906/3638366935.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


,season,week,month,home,score
1711,2019,1,8,0,0.500000
1702,2019,2,8,1,0.666667
1697,2019,3,8,1,3.000000
1682,2019,4,8,0,0.500000
1668,2019,5,9,1,1.000000
...,...,...,...,...,...
42,2023,16,12,1,2.000000
31,2023,17,12,0,1.500000
27,2023,18,12,1,1.000000
15,2023,19,12,0,0.750000
